In [3]:
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from recommender import recommender
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import creds  # Import

In [4]:
drake_sad = pd.read_csv('playlist_1.csv')
drake_hype = pd.read_csv('playlist_2.csv')
drake_chill = pd.read_csv('playlist_3.csv')
drake_romantic = pd.read_csv('playlist_4.csv')
drake_party = pd.read_csv('playlist_5.csv')

1. Split data into train and test
2. Run the recommendations on each song in the playlist, add recommendations to a recommendation list
3. Check if the recommendations are accurate with the test data

In [5]:
drake_sad.head()

,track_uri,track_name,duration_ms,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,spotify:track:047fCsbO4NdmwCBn8pcUXl,Marvins Room,347227,0.492,0.260,9,-17.341,0.0921,0.6460,0.001780,0.0705,0.3120,111.519
1,spotify:track:2Gnsof1hvZzjE1xdLRpjtf,Over My Dead Body,272573,0.489,0.570,1,-10.291,0.3060,0.7590,0.000002,0.1790,0.5840,185.516
2,spotify:track:6Z01gUquJsjJC67uNWm6P0,Shot For Me,224720,0.566,0.465,2,-11.714,0.2350,0.0555,0.000000,0.5670,0.1770,143.015
3,spotify:track:2KvHC9z14GSl4YpkNMX384,Do Not Disturb,283551,0.618,0.693,7,-5.943,0.4500,0.2460,0.000000,0.1120,0.4540,170.982
4,spotify:track:4wVOKKEHUJxHCFFNUWDn0B,Chicago Freestyle (feat. Giveon),220488,0.735,0.449,10,-7.507,0.3470,0.6290,0.000000,0.1130,0.0397,122.947


In [28]:
playlist_list = [drake_sad, drake_hype, drake_chill, drake_romantic, drake_party]

In [6]:
# Define a function to split the dataset into training and testing
def split_dataset(df, train_ratio=0.7):
    num_rows = len(df)
    num_train = int(num_rows * train_ratio)
    
    # Shuffle the DataFrame rows
    shuffled_df = df.sample(frac=1, random_state=42)
    
    # Split into training and testing DataFrames
    train_df = shuffled_df.iloc[:num_train]
    test_df = shuffled_df.iloc[num_train:]
    
    return train_df, test_df


In [30]:
# Splitting data
sad_train, sad_test = split_dataset(drake_sad)
hype_train, hype_test = split_dataset(drake_hype)
chill_train, chill_test = split_dataset(drake_chill)
romantic_train, romantic_test = split_dataset(drake_romantic)
party_train, party_test = split_dataset(drake_party)

In [9]:
DATASET_NAME = 'drake_songs_dataset.csv'

#Read in data as a dataframe
drake_df = pd.read_csv(DATASET_NAME)

#Get desired audio features
selected_features = [
    'danceability', 'energy', 'key', 'loudness',
    'speechiness', 'acousticness', 'instrumentalness', 
    'liveness', 'valence', 'tempo'
]



In [10]:
#Scales all data before computing cosine similarity matrix
def scale_data(input_song, drake_df):
    # Making a copy to not alter drake_df
    recommender_dataset = drake_df.copy()
    
    # Removing input song from recommender_dataset so it isn't recommended
    recommender_dataset = recommender_dataset[recommender_dataset['track_uri'] != input_song['track_uri']]

    #Getting only necessary columns before concat
    recommender_dataset = recommender_dataset[selected_features].copy()
    input_song = input_song[selected_features].copy().to_frame().T
    
    #Combining rows for features scaling
    all_features = pd.concat([input_song, recommender_dataset])
    scaler = StandardScaler()
    all_features_scaled = scaler.fit_transform(all_features)

    user_features = all_features_scaled[:1, :].copy()
    dataset_features = all_features_scaled[1:, :].copy()
    
    return user_features, dataset_features
    
        

In [11]:
# Function that runs the recommendation system
def make_recs(input_song_index, playlist_df, drake_df):
    #Keeps all columns so that we can extract the recommended song names and artists later
    df_all_cols = drake_df.copy()

    # Get the input song that we will make recommendations from
    input_song = playlist_df.iloc[input_song_index]

    #Remove the user's inputted track from original dataset so it isn't recommended later on
    drake_df =  drake_df[drake_df['track_uri'] != input_song['track_uri']]

    #Scale data
    user_features, dataset_features = scale_data(input_song, drake_df)

    # Recommending system
    return recommender(user_features, dataset_features, df_all_cols, 5)
    

In [38]:
def calculate_accuracy(recommended_songs, test_songs):
    # Convert the recommended and test songs to sets for efficient comparison
    recommended_set = set(recommended_songs)
    test_set = set(test_songs)
    
    # Count the number of correct recommendations (intersection of sets)
    num_correct_recommendations = len(recommended_set & test_set)
    
    # Calculate accuracy
    accuracy = num_correct_recommendations / len(test_set)
    
    return accuracy

def calculate_precision(recommended_songs, test_songs):
    recommended_set = set(recommended_songs)
    test_set = set(test_songs)
    
    true_positives = len(recommended_set & test_set)
    false_positives = len(recommended_set - test_set)
    
    precision = true_positives / (true_positives + false_positives)
    return precision

def calculate_recall(recommended_songs, test_songs):
    recommended_set = set(recommended_songs)
    test_set = set(test_songs)
    
    true_positives = len(recommended_set & test_set)
    false_negatives = len(test_set - recommended_set)
    
    recall = true_positives / (true_positives + false_negatives)
    return recall



In [50]:
NUM_PLAYLISTS = 5

#Empty lists to calculate average metrics later
accuracy_sum = 0
precision_sum = 0
recall_sum = 0

# Run the recommendation system on every song in the list for each playlist
for j, playlist in enumerate(playlist_list):
    #Splitting into training and testing data
    training_data, testing_data = split_dataset(playlist)

    # Create empty list to store recommendations
    recommendations = []

    # Running recommendation system
    for i, row in playlist.iterrows():
        recommendations += make_recs(i, playlist, drake_df)['track_name'].to_list()

    # Outputting accuracy for playlist
    print(f"Playlist {j + 1}")
    print("-------------------")    
    print(f"Accuracy: {round(calculate_accuracy(recommendations, testing_data['track_name']),2 )}\nPrecision: {round(calculate_precision(recommendations, testing_data['track_name']), 2)}\nRecall: {round(calculate_recall(recommendations, testing_data['track_name']), 2)}")
    print("-------------------")    

    #Storing metrics for later
    accuracy_sum += round(calculate_accuracy(recommendations, testing_data['track_name']), 2)
    precision_sum += round(calculate_accuracy(recommendations, testing_data['track_name']), 2)
    recall_sum += round(calculate_accuracy(recommendations, testing_data['track_name']), 2)

# Aggregate metrics
print(f"Mean Accuracy: {accuracy_sum / NUM_PLAYLISTS}")
print(f"Mean Precision: {precision_sum / NUM_PLAYLISTS}")
print(f"Mean Recall: {recall_sum / NUM_PLAYLISTS}")

Playlist 1
-------------------
Accuracy: 0.55
Precision: 0.05
Recall: 0.55
-------------------
Playlist 2
-------------------
Accuracy: 0.56
Precision: 0.08
Recall: 0.56
-------------------
Playlist 3
-------------------
Accuracy: 0.69
Precision: 0.16
Recall: 0.69
-------------------
Playlist 4
-------------------
Accuracy: 0.75
Precision: 0.22
Recall: 0.75
-------------------
Playlist 5
-------------------
Accuracy: 0.74
Precision: 0.26
Recall: 0.74
-------------------
Mean Accuracy: 0.658
Mean Precision: 0.658
Mean Recall: 0.658
